# Eval the results of TODO

In [1]:
import pandas as pd
from datasets import load_dataset
from sklearn.metrics import f1_score, precision_score, recall_score, \
    accuracy_score

In [4]:

# load
model = 'Starling-LM-7B-alpha'

# load results
#zero_shot = pd.read_csv(f"../zero-shot/data/{model}.csv")
#zero_shot_with_system = pd.read_csv(
#    f"../zero-shot-system_prompt/data/{model}.csv")
#zero_shot_cot = pd.read_csv(f"../zero-shot-cot/data/{model}.csv")
#two_shot = pd.read_csv(f"../2-shot/data/{model}.csv")
#four_shot = pd.read_csv(f"../4-shot/data/{model}.csv")
#eight_shot = pd.read_csv(f"../8-shot/data/{model}.csv")
#
#two_shot_cot = pd.read_csv(f"../2-shot-CoT/data/{model}.csv")
#four_shot_cot = pd.read_csv(f"../4-shot-CoT/data/{model}.csv")
eight_shot_cot = pd.read_csv(f"../8-shot-CoT/data/{model}.csv")

#load pool
pool = load_dataset('mediabiasgroup/BABE-icl-pool')['train'].to_pandas()

# exclude pool from model (if needed)
#zero_shot = zero_shot.merge(pool['text'], on='text', how='left',
#                            indicator=True).query(
#    '_merge == "left_only"').drop('_merge', axis=1)
#zero_shot_with_system = zero_shot_with_system.merge(pool['text'], on='text',
#                                                    how='left',
#                                                    indicator=True).query(
#    '_merge == "left_only"').drop('_merge', axis=1)
#zero_shot_cot = zero_shot_cot.merge(pool['text'], on='text', how='left',
#                                    indicator=True).query(
#    '_merge == "left_only"').drop('_merge', axis=1)
#two_shot = two_shot.merge(pool['text'], on='text', how='left',
#                          indicator=True).query('_merge == "left_only"').drop(
#    '_merge', axis=1)
#four_shot = four_shot.merge(pool['text'], on='text', how='left',
#                            indicator=True).query(
#    '_merge == "left_only"').drop('_merge', axis=1)
#eight_shot = eight_shot.merge(pool['text'], on='text', how='left',
#                              indicator=True).query(
#    '_merge == "left_only"').drop('_merge', axis=1)
#two_shot_cot = two_shot_cot.merge(pool['text'], on='text', how='left',
#                              indicator=True).query(
#    '_merge == "left_only"').drop('_merge', axis=1)
#four_shot_cot = four_shot_cot.merge(pool['text'], on='text', how='left',
#                              indicator=True).query(
#    '_merge == "left_only"').drop('_merge', axis=1)
eight_shot_cot = eight_shot_cot.merge(pool['text'], on='text', how='left',
                              indicator=True).query(
    '_merge == "left_only"').drop('_merge', axis=1)


#load babe
dataset = load_dataset('mediabiasgroup/BABE-v4')
df_babe = pd.DataFrame(dataset['train'])

# df_merge = babe at begin
df_merge_all = df_babe

def find_first_occurrence(string, instruction="Instruction:",
                          statement1="The answer is BIASED",
                          statement2="The answer is NOT BIASED"
                          ):
    index_instruction = string.find(instruction)
    index1 = string.find(statement1)
    index2 = string.find(statement2)

    if index1 == -1 and index2 == -1:
        return "Neither instruction nor statements found in the given string."
    elif index1 == -1 and index2 == -1:
        return "Neither statement found in the given string."
    elif index1 == -1:
        if index_instruction == -1 or index2 < index_instruction:
            return f"{statement2}"
        else:
            return "After Instruction"
    elif index2 == -1:
        if index_instruction == -1 or index1 < index_instruction:
            return f"{statement1}"
        else:
            return "After Instruction"
    elif index1 < index2:
        if index_instruction == -1 or index1 < index_instruction:
            return f"{statement1}"
        else:
            return "After Instruction"
    else:
        if index_instruction == -1 or index2 < index_instruction:
            return f"{statement2}"
        else:
            return "After Instruction"



# Zero shot

In [1]:
zero_shot.query("label == '?'")

NameError: name 'zero_shot' is not defined

In [ ]:
# preprocessing

In [ ]:
zero_shot = zero_shot.rename(columns={"label": "0_shot_label"})
zero_shot['0_shot_label'] = zero_shot['0_shot_label'].replace('BIASED', 1)
zero_shot['0_shot_label'] = zero_shot['0_shot_label'].replace('NOT BIASED', 0)

df_merge = pd.merge(df_babe, zero_shot[['text', '0_shot_label']], on='text')
ground_truth = df_merge['label']
zero_shot_label = df_merge['0_shot_label']

In [ ]:
print("F1-Score with TODO: ", f1_score(ground_truth, zero_shot_label))
print("Precision with TODO: ",
      precision_score(ground_truth, zero_shot_label))
print("Recall with TODO: ",
      recall_score(ground_truth, zero_shot_label))
print("Accuracy with TODO: ",
      accuracy_score(ground_truth, zero_shot_label))

# zero shot with system prompt

In [ ]:
zero_shot_with_system.query("label == '?'")

In [ ]:
zero_shot_with_system = zero_shot_with_system.rename(
    columns={"label": "0_shot_with_system_label"})
zero_shot_with_system['0_shot_with_system_label'] = zero_shot_with_system[
    '0_shot_with_system_label'].replace('BIASED', 1)
zero_shot_with_system['0_shot_with_system_label'] = zero_shot_with_system[
    '0_shot_with_system_label'].replace('NOT BIASED', 0)

df_merge = pd.merge(df_merge, zero_shot_with_system[
    ['text', '0_shot_with_system_label']], on='text')
ground_truth = df_merge['label']
zero_shot_with_system_label = df_merge['0_shot_with_system_label']

In [ ]:
print("F1-Score with TODO with System Prompt: ",
      f1_score(ground_truth, zero_shot_with_system_label))
print("Precision with TODO with System Prompt: ",
      precision_score(ground_truth, zero_shot_with_system_label))
print("Recall with TODO with System Prompt: ",
      recall_score(ground_truth, zero_shot_with_system_label))
print("Accuracy with TODO with System Prompt: ",
      accuracy_score(ground_truth, zero_shot_with_system_label))

# zero shot CoT

In [ ]:
zero_shot_cot.query("label == '?'")

In [ ]:
# preprocessing

In [ ]:
zero_shot_cot = zero_shot_cot.rename(columns={"label": "0_shot_cot_label"})
zero_shot_cot['0_shot_cot_label'] = zero_shot_cot['0_shot_cot_label'].replace(
    'BIASED', 1)
zero_shot_cot['0_shot_cot_label'] = zero_shot_cot['0_shot_cot_label'].replace(
    'NOT BIASED', 0)

df_merge = pd.merge(df_merge, zero_shot_cot[['text', '0_shot_cot_label']],
                    on='text')
ground_truth = df_merge['label']
zero_shot_cot_label = df_merge['0_shot_cot_label']

In [ ]:
print("F1-Score with TODO with CoT: ",
      f1_score(ground_truth, zero_shot_cot_label))
print("Precision with TODO with CoT: ",
      precision_score(ground_truth, zero_shot_cot_label))
print("Recall with TODO with CoT: ",
      recall_score(ground_truth, zero_shot_cot_label))
print("Accuracy with TODO with CoT: ",
      accuracy_score(ground_truth, zero_shot_cot_label))

# 2-shot

In [ ]:
two_shot.query("label == '?'")

In [ ]:
# preprocessing

In [2]:
two_shot = two_shot.rename(columns={"label": "2_shot_label"})
two_shot['2_shot_label'] = two_shot['2_shot_label'].replace('BIASED', 1)
two_shot['2_shot_label'] = two_shot['2_shot_label'].replace('NOT BIASED', 0)

df_merge = pd.merge(df_merge, two_shot[['text', '2_shot_label']], on='text')
ground_truth = df_merge['label']
two_shot_label = df_merge['2_shot_label']

NameError: name 'two_shot' is not defined

In [ ]:
print("F1-Score with TODO with (2 shot): ",
      f1_score(ground_truth, two_shot_label))
print("Precision with TODO with (2 shot): ",
      precision_score(ground_truth, two_shot_label))
print("Recall with TODO with (2 shot): ",
      recall_score(ground_truth, two_shot_label))
print("Accuracy with TODO with (2 shot): ",
      accuracy_score(ground_truth, two_shot_label))

# 4-shot

In [ ]:
four_shot.query("label == '?'")

In [ ]:
# preprocessing

In [ ]:
four_shot = four_shot.rename(columns={"label": "4_shot_label"})
four_shot['4_shot_label'] = four_shot['4_shot_label'].replace('BIASED', 1)
four_shot['4_shot_label'] = four_shot['4_shot_label'].replace('NOT BIASED', 0)

df_merge = pd.merge(df_merge, four_shot[['text', '4_shot_label']], on='text')
ground_truth = df_merge['label']
four_shot_label = df_merge['4_shot_label']

In [ ]:
print("F1-Score with TODO with (4 shot): ",
      f1_score(ground_truth, four_shot_label))
print("Precision with TODO with (4 shot): ",
      precision_score(ground_truth, four_shot_label))
print("Recall with TODO with (4 shot): ",
      recall_score(ground_truth, four_shot_label))
print("Accuracy with TODO with (4 shot): ",
      accuracy_score(ground_truth, four_shot_label))

# 8-shot

In [ ]:
eight_shot.query("label == '?'")

In [ ]:
# preprocessing

In [ ]:
eight_shot = eight_shot.rename(columns={"label": "8_shot_label"})
eight_shot['8_shot_label'] = eight_shot['8_shot_label'].replace('BIASED', 1)
eight_shot['8_shot_label'] = eight_shot['8_shot_label'].replace('NOT BIASED',
                                                                0)

df_merge = pd.merge(df_merge, eight_shot[['text', '8_shot_label']], on='text')
ground_truth = df_merge['label']
eight_shot_label = df_merge['8_shot_label']

In [ ]:
print("F1-Score with TODO with (8 shot): ",
      f1_score(ground_truth, eight_shot_label))
print("Precision with TODO with (8 shot): ",
      precision_score(ground_truth, eight_shot_label))
print("Recall with TODO with (8 shot): ",
      recall_score(ground_truth, eight_shot_label))
print("Accuracy with TODO with (8 shot): ",
      accuracy_score(ground_truth, eight_shot_label))

# 2-shot CoT

In [ ]:
two_shot_cot.query("label == '?'")

In [ ]:
# preprocessing

In [ ]:
two_shot_cot = two_shot_cot.rename(columns={"label": "2_shot_cot_label"})
two_shot_cot['2_shot_cot_label'] = two_shot_cot['2_shot_cot_label'].replace('BIASED', 1)
two_shot_cot['2_shot_cot_label'] = two_shot_cot['2_shot_cot_label'].replace('NOT BIASED',
                                                                0)

df_merge = pd.merge(df_merge, two_shot_cot[['text', '2_shot_cot_label']], on='text')
ground_truth = df_merge['label']
two_shot_cot_label = df_merge['2_shot_cot_label']

In [ ]:
print("F1-Score with TODO with (2 shot CoT): ",
      f1_score(ground_truth, two_shot_cot_label))
print("Precision with TODO with (2 shot CoT): ",
      precision_score(ground_truth, two_shot_cot_label))
print("Recall with TODO with (2 shot CoT): ",
      recall_score(ground_truth, two_shot_cot_label))
print("Accuracy with TODO with (2 shot CoT): ",
      accuracy_score(ground_truth, two_shot_cot_label))

# 4-shot CoT

In [ ]:
four_shot_cot.query("label == '?'")

In [ ]:
# preprocessing

In [ ]:
four_shot_cot = four_shot_cot.rename(columns={"label": "4_shot_cot_label"})
four_shot_cot['4_shot_cot_label'] = four_shot_cot['4_shot_cot_label'].replace('BIASED', 1)
four_shot_cot['4_shot_cot_label'] = four_shot_cot['4_shot_cot_label'].replace('NOT BIASED',
                                                                0)

df_merge = pd.merge(df_merge, four_shot_cot[['text', '4_shot_cot_label']], on='text')
ground_truth = df_merge['label']
four_shot_cot_label = df_merge['4_shot_cot_label']

In [ ]:
print("F1-Score with TODO with (4 shot CoT): ",
      f1_score(ground_truth, four_shot_cot_label))
print("Precision with TODO with (4 shot CoT): ",
      precision_score(ground_truth, four_shot_cot_label))
print("Recall with TODO with (4 shot CoT): ",
      recall_score(ground_truth, four_shot_cot_label))
print("Accuracy with TODO with (4 shot CoT): ",
      accuracy_score(ground_truth, four_shot_cot_label))

# 8-shot CoT

In [3]:
eight_shot_cot.query("label == '?'")

,text,response,raw_data,query,label
0,But instead of closing loopholes that allow cr...,The sentence is biased because it uses emotion...,You are an expert in media bias.\n\nInstructio...,You are an expert in media bias.\n\nInstructio...,?
1,"""You think I'm joking,"" he continued. """,The sentence is a direct quote from an individ...,You are an expert in media bias.\n\nInstructio...,You are an expert in media bias.\n\nInstructio...,?
2,"Yes, the rich are rich and want to stay that w...",The sentence is biased because it presents a n...,You are an expert in media bias.\n\nInstructio...,You are an expert in media bias.\n\nInstructio...,?
3,Americans are carrying $1.57 trillion in outst...,The sentence presents a factual statement abou...,You are an expert in media bias.\n\nInstructio...,You are an expert in media bias.\n\nInstructio...,?
4,Republican state legislators from Texas to Ari...,The sentence is biased because it employs emot...,You are an expert in media bias.\n\nInstructio...,You are an expert in media bias.\n\nInstructio...,?
...,...,...,...,...,...
4016,Far-right talk show host Glenn Beck is being s...,"The sentence is biased because it uses strong,...",You are an expert in media bias.\n\nInstructio...,You are an expert in media bias.\n\nInstructio...,?
4017,After President Donald Trump’s news conference...,The sentence is biased because it accuses the ...,You are an expert in media bias.\n\nInstructio...,You are an expert in media bias.\n\nInstructio...,?
4018,When Republicans warned Democrats that impeach...,The sentence is biased because it uses emotion...,You are an expert in media bias.\n\nInstructio...,You are an expert in media bias.\n\nInstructio...,?
4019,Sen. Tom Cotton (R-AR) says it “makes absolute...,The sentence is biased because it uses loaded ...,You are an expert in media bias.\n\nInstructio...,You are an expert in media bias.\n\nInstructio...,?


In [5]:
# preprocessing
def update_label(row):
    if row['response'].startswith('BIASED') and row['label'] == '?':
        return 'BIASED'
    elif row['response'].startswith('NOT BIASED') and row[
        'label'] == '?':
        return 'NOT BIASED'
    elif row['response'].startswith('Classification: NOT BIASED') and row[
        'label'] == '?':
        return 'NOT BIASED'
    elif row['response'].startswith('Classification: BIASED') and row[
        'label'] == '?':
        return 'BIASED'
    elif row['response'].startswith('Classify the sentence above as BIASED or NOT BIASED.\n\nOutput: NOT BIASED') and row[
        'label'] == '?':
        return 'NOT BIASED'
    elif row['response'].startswith('Classify the sentence above as BIASED or NOT BIASED.\n\nOutput: BIASED') and row[
        'label'] == '?':
        return 'BIASED'
    elif find_first_occurrence(row['response']) == "The answer is BIASED"and row[
        'label'] == '?':
        return 'BIASED'
    elif find_first_occurrence(row['response']) == "The answer is NOT BIASED"and row[
        'label'] == '?':
        return 'NOT BIASED'
    else:
        return row['label']


eight_shot_cot['label'] = eight_shot_cot.apply(update_label, axis=1)
eight_shot_cot.query("label == '?'")

,text,response,raw_data,query,label


In [8]:
eight_shot_cot = eight_shot_cot.rename(columns={"label": "8_shot_cot_label"})
eight_shot_cot['8_shot_cot_label'] = eight_shot_cot['8_shot_cot_label'].replace('BIASED', 1)
eight_shot_cot['8_shot_cot_label'] = eight_shot_cot['8_shot_cot_label'].replace('NOT BIASED',
                                                                0)
                                                                
df_merge = pd.merge(df_babe, eight_shot_cot[['text', '8_shot_cot_label']], on='text')
ground_truth = df_merge['label']
eight_shot_cot_label = df_merge['8_shot_cot_label']

In [9]:
print("F1-Score with TODO with (8 shot CoT): ",
      f1_score(ground_truth, eight_shot_cot_label))
print("Precision with TODO with (8 shot CoT): ",
      precision_score(ground_truth, eight_shot_cot_label))
print("Recall with TODO with (8 shot CoT): ",
      recall_score(ground_truth, eight_shot_cot_label))
print("Accuracy with TODO with (8 shot CoT): ",
      accuracy_score(ground_truth, eight_shot_cot_label))

F1-Score with TODO with (8 shot CoT):  0.7694425326909842
Precision with TODO with (8 shot CoT):  0.7081925675675675
Recall with TODO with (8 shot CoT):  0.8422903063787042
Accuracy with TODO with (8 shot CoT):  0.7500621735886596


# Comparison and plots

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


def plot_confusion_matrix(ax, df, true_labels_column, predicted_labels_column,
                          title=None
                          ):
    predicted_labels = df[f'{predicted_labels_column}']
    true_labels = df[f'{true_labels_column}']

    # Calculate confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)


    # Display confusion matrix heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=True,
                yticklabels=True, ax=ax)

    title = title if title else predicted_labels_column

    ax.set_title(f'Confusion Matrix - {title}')
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')


# Create subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Confusion Matrices')

# Plot each confusion matrix
plot_confusion_matrix(axes[0, 0], df_merge, 'label', '0_shot_label', '0_shot')
plot_confusion_matrix(axes[0, 1], df_merge, 'label',
                      '0_shot_with_system_label', '0_shot_with_system')
plot_confusion_matrix(axes[0, 2], df_merge, 'label', '0_shot_cot_label',
                      '0_shot_cot')
plot_confusion_matrix(axes[1, 0], df_merge, 'label', '2_shot_label', '2_shot')
plot_confusion_matrix(axes[1, 1], df_merge, 'label', '4_shot_label', '4_shot')
plot_confusion_matrix(axes[1, 2], df_merge, 'label', '8_shot_label', '8_shot')

plt.tight_layout(
    rect=[0, 0, 1, 0.96])  # Adjust layout to prevent title overlap
plt.show()

# Krippendorff Alpha in model

In [ ]:
import krippendorff

runs = ['0_shot_label', '0_shot_with_system_label', '0_shot_cot_label',
        '2_shot_label', '4_shot_label', '8_shot_label']


def compute_krippendorff_alpha(df, predicted_columns):
    pred_map = {}
    for run in predicted_columns:
        predicted_labels = df[run]
        pred_map[run] = predicted_labels

    # Check if there is variability in the ratings
    unique_labels_counts = df[predicted_columns].nunique(axis=1)
    if unique_labels_counts.max() == 1:
        # All ratings are the same, return a special value or handle accordingly
        return 0

    reliability_data = df[predicted_columns].values.tolist()

    # Calculate Krippendorff's alpha
    alpha = krippendorff.alpha(reliability_data=list(pred_map.values()),
                               level_of_measurement='nominal')

    return alpha


In [ ]:
alpha_value = compute_krippendorff_alpha(df_merge, runs)
print(f"Krippendorff's Alpha (all runs): {alpha_value}")

In [ ]:
import itertools

def compute_krippendorff_alpha_for_k_runs(df, runs, k=None):
    # Initialize variables to store the best combination and alpha
    if k is None:
        k = len(runs)

    best_combination = None
    best_alpha = 0  # Assuming alpha ranges from 0 to 1

    # Iterate through all possible combinations
    for combination in itertools.combinations(runs, k):

        alpha_value = compute_krippendorff_alpha(df, list(combination))

        # Print alpha for the current combination
        print(f"Combination: {combination}, Alpha: {alpha_value}")

        # Update best combination and alpha if a higher alpha is found
        if alpha_value > best_alpha:
            best_alpha = alpha_value
            best_combination = combination

    # Print the best combination and alpha
    print(f"\nBest Combination: {best_combination}, Best Alpha: {best_alpha}")
    return best_alpha, best_combination

In [ ]:
compute_krippendorff_alpha_for_k_runs(df_merge, runs, 2)